# BentoML {{cookiecutter.framework}} {{cookiecutter.project_name}} Tutorial

[Source](https://github.com/bentoml/gallery/blob/main/{{ cookiecutter.__project_dir }}/{{ cookiecutter.__project_slug }}).
Try it out on [Colab](https://colab.research.google.com/github/bentoml/gallery/blob/main/{{ cookiecutter.__project_dir }}/{{ cookiecutter.__project_slug }}/{{ cookiecutter.__full_name }}_demo.ipynb).

Install required dependencies:

In [ ]:
!pip install -r requirements.txt

## Define the model

First let's define a simple {{cookiecutter.framework}} model

In [ ]:
# Contributed by {{cookiecutter.author}}.
# {{cookiecutter.framework}} {{cookiecutter.project_name}} model implementation.

import {{ cookiecutter.__project_dir.replace('pytorch', 'torch') }}

## Training and Saving the model

Then we define some helper functions that can be used for training:

In [ ]:
import os
import random

import bentoml
import numpy as np
from sklearn.model_selection import KFold
#import necessary library for training here

# reproducible setup for testing
seed = 42
random.seed(seed)
np.random.seed(seed)

In [ ]:
K_FOLDS = 5
NUM_EPOCHS = 5
LOSS_FUNCTION = ... # callable loss function

def get_dataset(): ...


def train_epoch(model, optimizer, loss_function, train_loader, epoch, device="cpu"): ...


def test_model(model, test_loader, device="cpu"): ...

In [ ]:
# load data
train_set, test_set = get_dataset()
test_loader = ... 

### Cross Validation

We can do some cross validation and the results can be saved with the model as metadata


In [ ]:
def cross_validate(dataset, epochs=NUM_EPOCHS, k_folds=K_FOLDS): ... 

In [ ]:
cv_results = cross_validate(train_set, epochs=1)

### Training the model

In [ ]:
def train(dataset, epochs=NUM_EPOCHS, device="cpu"): ...

In [ ]:
trained_model = train(train_set)

### Saving the model with some metadata

In [ ]:
correct, total = test_model(trained_model, test_loader)
metadata = {
    "accuracy": float(correct)/total,
    "cv_stats": cv_results,
}

tag = bentoml.{{ cookiecutter.__project_dir }}.save(
    "{{ cookiecutter.__full_name }}",
    trained_model,
    metadata=metadata,
)

## Create a BentoML Service for serving the model


Note: using `%%writefile` here because `bentoml.Service` instance must be created in a separate `.py` file

Even though we have only one model, we can create as many API endpoints as we want. Here we create endpoints...

In [ ]:
%%writefile service.py
import typing as t

import bentoml
import numpy as np
import PIL.Image

from bentoml.io import Image, NumpyNdarray
from PIL.Image import Image as PILImage

{{ cookiecutter.__project_slug }}_runner = bentoml.{{ cookiecutter.__project_dir }}.load_runner(
    "{{ cookiecutter.__full_name }}",
    name="{{ cookiecutter.__project_slug }}_runner",
)

svc = bentoml.Service(
    name="{{ cookiecutter.__full_name }}",
    runners=[
        {{ cookiecutter.__project_slug }}_runner,
    ],
)

Start a dev model server to test out the service defined above

In [ ]:
!bentoml serve service.py:svc

Now you can use something like:

```bash
$ curl -H "Content-Type: multipart/form-data" -F'fileobj=@samples/1.png;type=image/png' http://127.0.0.1:5000/predict
```
    
to send an image to the digit recognition service

## Build a Bento for distribution and deployment
There are two ways to build a BentoML bundle:

One can define a `bentofile.yaml`:

```yaml
service: "service:svc"
description: "file: ./README.md"
labels:
  owner: {{cookiecutter.author.lower().replace(' ', '-')}}
  stage: demo
include:
- "*.py"
exclude:
- "tests/"
python:
  packages:
    - scikit-learn
    - Pillow
```

Then run `bentoml build` in a terminal session in this directory:

```bash
$ bentoml build
```

One can also use BentoML's python API `bentoml.build`: 

In [ ]:
bentoml.build(
    "service.py:svc",
    include=["*.py"],
    exclude=["tests/"],
    description="file:./README.md",
    python=dict(
        packages=["scikit-learn", "torch", "Pillow"],
    )
)

Starting a development server with the new bento with `bentoml serve`:

```bash
$ bentoml serve {{ cookiecutter.__full_name }}:latest
```